# Attempt 1 at Optimizing

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns={'EIN', 'NAME'})

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_numbers = application_df['APPLICATION_TYPE'].value_counts()
app_numbers

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace= list(app_numbers[app_numbers < 1000].index)

In [6]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_numbers = application_df["CLASSIFICATION"].value_counts()
classification_numbers

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C4200        1
C4500        1
C1283        1
C2600        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_numbers = classification_numbers[classification_numbers >= 1]
classification_numbers

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C4200        1
C4500        1
C1283        1
C2600        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_numbers[classification_numbers < 1800].index)

In [10]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")
    
# Check to make sure bucketing was successful
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df=pd.get_dummies(application_df)


In [13]:
# Split our preprocessed data into our features and target arrays
X=dummy_df.drop(columns={'IS_SUCCESSFUL'}, axis=1)
y=dummy_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                410       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              period=5,
                              )

In [18]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 722us/step - loss: 32010.7656 - accuracy: 0.5133
Epoch 2/50
804/804 [==============================] - 1s 979us/step - loss: 3185.1741 - accuracy: 0.4933
Epoch 3/50
804/804 [==============================] - 1s 799us/step - loss: 2088.1316 - accuracy: 0.5159
Epoch 4/50
804/804 [==============================] - 1s 947us/step - loss: 3297.5388 - accuracy: 0.5033
Epoch 5/50
776/804 [===========================>..] - ETA: 0s - loss: 3455.7976 - accuracy: 0.5014
Epoch 00005: saving model to checkpoints\weights.5.hdf5
804/804 [==============================] - 1s 818us/step - loss: 3338.5149 - accuracy: 0.5002
Epoch 6/50
804/804 [==============================] - 1s 782us/step - loss: 1441.2395 - accuracy: 0.4921
Epoch 7/50
804/804 [==============================] - 1s 800us/step - loss: 453.2607 - accuracy: 0.5116
Epoch 8/50
804/804 [==============================] - 1s 880us/step - loss: 170.9235 - accuracy: 0.5212
Epoch 9/50
804/80

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.9326 - accuracy: 0.5326 - 251ms/epoch - 936us/step
Loss: 0.9326077699661255, Accuracy: 0.5325947403907776


# Attempt 2 at Optimizing

In [20]:
# Split our preprocessed data into our features and target arrays
X2=dummy_df.drop(columns={'IS_SUCCESSFUL'}, axis=1)
y2=dummy_df['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=14)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler2 = scaler.fit(X_train2)

# Scale the data
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features2 = len(X_train_scaled2[0])
hidden_nodes_layer12 = 10
hidden_nodes_layer22 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer12, input_dim=number_input_features2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer22, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                410       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 531
Trainable params: 531
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer2, metrics=["accuracy"])

In [24]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              period=5,
                              )

In [25]:
# Train the model
fit_model2 = nn.fit(X_train2,y_train2,epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 5711.9810 - accuracy: 0.4944
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6973 - accuracy: 0.4702
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6918 - accuracy: 0.5323
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 5/100
800/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5323
Epoch 00005: saving model to checkpoints\weights.5.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 9/100
804/804 [===========================

804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 65/100
775/804 [===========================>..] - ETA: 0s - loss: 0.6911 - accuracy: 0.5322
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 66/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 68/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 70/100
795/804 [============================>.] - ETA: 0s - loss: 0.6911 - accuracy: 0.5319
Epoch 00070: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6911 - accuracy: 0.5323
Epoch 71/100
80

In [26]:
# Evaluate the model using the test data
model_loss2, model_accuracy2 = nn.evaluate(X_test_scaled2,y_test2,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

268/268 - 0s - loss: 0.8152 - accuracy: 0.5326 - 218ms/epoch - 812us/step
Loss: 0.8152142763137817, Accuracy: 0.5325947403907776


# Attempt 3 at Optimizing

In [27]:
dummy_df_3=dummy_df.drop(columns={'ASK_AMT'}, axis=1)

In [28]:
# Split our preprocessed data into our features and target arrays
X3=dummy_df_3.drop(columns={'IS_SUCCESSFUL'}, axis=1)
y3=dummy_df_3['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, random_state=14)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler3 = scaler.fit(X_train3)

# Scale the data
X_train_scaled3 = X_scaler3.transform(X_train3)
X_test_scaled3 = X_scaler3.transform(X_test3)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled3[0])
hidden_nodes_layer13 = 80
hidden_nodes_layer23 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer13, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer23, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3200      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,661
Trainable params: 5,661
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
optimizer3 = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer3, metrics=["accuracy"])

In [32]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              period=5,
                              )

In [33]:
# Train the model
fit_model3 = nn.fit(X_train3,y_train3,epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5727 - accuracy: 0.7196
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7315
Epoch 3/100
804/804 [==============================] - 1s 946us/step - loss: 0.5533 - accuracy: 0.7329
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5515 - accuracy: 0.7313
Epoch 5/100
804/804 [==============================] - ETA: 0s - loss: 0.5502 - accuracy: 0.7334
Epoch 00005: saving model to checkpoints\weights.5.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5489 - accuracy: 0.7341
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7338
Epoch 9/100
804/804 [============================

804/804 [==============================] - 1s 1ms/step - loss: 0.5387 - accuracy: 0.7401
Epoch 65/100
763/804 [===========================>..] - ETA: 0s - loss: 0.5386 - accuracy: 0.7397
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5386 - accuracy: 0.7403
Epoch 66/100
804/804 [==============================] - 1s 917us/step - loss: 0.5387 - accuracy: 0.7398
Epoch 67/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7400
Epoch 68/100
804/804 [==============================] - 1s 895us/step - loss: 0.5382 - accuracy: 0.7399
Epoch 69/100
804/804 [==============================] - 1s 926us/step - loss: 0.5383 - accuracy: 0.7411
Epoch 70/100
779/804 [============================>.] - ETA: 0s - loss: 0.5389 - accuracy: 0.7399
Epoch 00070: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7402
Epoch 71/

In [34]:
# Evaluate the model using the test data
model_loss3, model_accuracy3 = nn.evaluate(X_test_scaled3,y_test3,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

268/268 - 0s - loss: 4.4689 - accuracy: 0.6042 - 267ms/epoch - 996us/step
Loss: 4.468893527984619, Accuracy: 0.6041982769966125
